# Setup of Noteboook

The follwing code clones the github repository with course files. 
Subsequently it imports all libraries and custom modules needed for this notebook

In [ ]:
!git clone https://github.com/japolak/datahow-course-scripts.git
!pip install --upgrade scipy==1.7.3
!cd /content/datahow-course-scripts

In [89]:
# import libaries
import pandas as pd
import numpy as np
import scipy
import importlib  
import scipy.integrate
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import pandas.util.testing as tm
import statsmodels.api as sm
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler
from sklearn.cross_decomposition import PLSRegression
from sklearn.metrics import mean_squared_error



# import custom modules
emulator = importlib.import_module("datahow-course-scripts.scripts.modules.emulator")
plothelpers = importlib.import_module("datahow-course-scripts.scripts.modules.plothelpers")

# Regression Modelling

In this section, we first create load the train experiments that were create inthe PCA script and save in the file "Experiment_Train_Set.mat". 
These experimental data will be used to create a MLR model to predict final titer, which will serve as term of comparison for the PLS model trained below.

# Initial Conditions model

We load the OWU dataset with DOE dataset for both train set and test set.

In [90]:
filename = "owu-test.csv"
owu = pd.read_csv("datahow-course-scripts/scripts/datasets/"+filename,index_col=None)
owu.columns =  ["X:VCD", "X:Glc", "X:Lac", "X:Titer","W:Feed"]
owu.index = pd.MultiIndex.from_product([list(range(250)),list(range(15))], names=["run","time"])
owu_test = owu

filename = "owu.csv"
owu = pd.read_csv("datahow-course-scripts/scripts/datasets/"+filename,index_col=None)
owu.columns =  ["X:VCD", "X:Glc", "X:Lac", "X:Titer","W:Feed"]
owu.index = pd.MultiIndex.from_product([list(range(100)),list(range(15))], names=["run","time"])
owu

X:VCD   X:Glc     X:Lac     X:Titer  W:Feed
run time                                                
0   0      0.65336  24.989    0.0000     0.00000   0.000
    1      1.99900  23.828    1.7454     0.12965   0.000
    2      5.31610  20.523    6.7136     2.26180  19.803
    3     10.93400  32.668   18.2210    21.07500  19.803
    4     16.31000  39.240   38.0940   100.25000  19.803
...            ...     ...       ...         ...     ...
99  10     7.07870  39.417  159.2100  1207.80000   0.000
    11     5.30480  33.518  168.0700  1303.90000   0.000
    12     3.94070  29.116  174.6800  1373.40000   0.000
    13     2.90660  25.859  179.5800  1422.30000   0.000
    14     2.13220  23.464  183.1800  1455.30000   0.000

[1500 rows x 5 columns]

In [91]:
filename = "bwu-test_doe_unscaled.csv"
doe = pd.read_csv("datahow-course-scripts/scripts/datasets/"+filename,index_col=None, header=None)
doe.columns=["feed_start","feed_end","Glc_feed_rate","Glc_0","VCD_0"]
doe_test = doe

filename = "bwu_doe_unscaled.csv"
doe = pd.read_csv("datahow-course-scripts/scripts/datasets/"+filename,index_col=None, header=None)
doe.columns=["feed_start","feed_end","Glc_feed_rate","Glc_0","VCD_0"]
doe

,feed_start,feed_end,Glc_feed_rate,Glc_0,VCD_0
0,-0.57173,-0.65234,0.98479,0.973680,0.229680
1,-0.30239,0.70694,0.82030,-0.001358,0.483670
2,0.54568,0.60900,-0.39029,-0.840300,0.774100
3,-0.84467,-0.51225,-0.63137,-0.676170,-0.440310
4,-0.83495,0.42833,-0.64174,-0.024642,-0.019367
...,...,...,...,...,...
95,-0.44449,0.21562,-0.30480,-0.245270,-0.847970
96,-0.60051,-0.88541,0.66473,-0.057922,0.150660
97,0.36934,0.41479,0.46114,-0.613140,0.190860
98,-0.95273,-0.54334,0.89206,0.016768,-0.958280


In [92]:
# Define Targets as Titer at day 15
y = pd.DataFrame(owu["X:Titer"][:,14])
y_test = pd.DataFrame(owu_test["X:Titer"][:,14])
y

,X:Titer
run,
0,1311.10
1,1056.10
2,993.78
3,415.18
4,846.94
...,...
95,951.19
96,1505.10
97,1371.70


## Multiple Linear Regression 

We first train (fit) a quadratic MLR model to the final titer, using the data loaded above.
We then characterize the performance of the model using ANOVA, where the user can have information about the importance of each coefficient and of the overal model.

### Create linear model using stepwise procedure

In [93]:
# Fit Linear Model
X = sm.add_constant(doe.values)
mod = sm.OLS(y.values, X)
res = mod.fit()
print(res.summary())

                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.178
Model:                            OLS   Adj. R-squared:                  0.134
Method:                 Least Squares   F-statistic:                     4.063
Date:                Fri, 22 Apr 2022   Prob (F-statistic):            0.00221
Time:                        05:52:17   Log-Likelihood:                -716.86
No. Observations:                 100   AIC:                             1446.
Df Residuals:                      94   BIC:                             1461.
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const       1105.8596     32.394     34.138      0.0

### Evaluate model prediction on train set
In this section, we check the predictions of the models versus the training data for the final titer.


In [94]:
reg = LinearRegression().fit(doe, y)
yhat = reg.predict(doe).reshape(len(y))
fig = px.scatter(x=y.values.reshape(len(yhat)),y=yhat,labels=dict(x="Observed Titer", y="Predicted Titer"), title="Observed vs Predicted on Train Set")
fig.update_layout(shapes = [{'type': 'line', 'yref': 'paper', 'xref': 'paper', 'y0': 0, 'y1': 1, 'x0': 0, 'x1': 1, 'layer': 'below'}])
fig.show()

### Evaluate model prediction on test set

In this section, we use the trained model to predict the final titer and we test it on the test set created in the script "00_Process_Characterization.ipynb".


In [95]:
yhat_test = reg.predict(doe_test).reshape(len(y_test))
fig = px.scatter(x=y_test.values.reshape(len(yhat_test)),y=yhat_test,labels=dict(x="Observed Titer", y="Predicted Titer"), title="Observed vs Predicted on Train Set")
fig.update_layout(shapes = [{'type': 'line', 'yref': 'paper', 'xref': 'paper', 'y0': 0, 'y1': 1, 'x0': 0, 'x1': 1, 'layer': 'below'}])
fig.show()

In [96]:
# Absolute RMSE
rmse_abs = mean_squared_error(y_test, yhat_test,squared=False)
print('Absolute RMSE: ',rmse_abs)
# Relative RMSE
rmse_rel = mean_squared_error(y_test, yhat_test,squared=False) / np.std(y_test.values)
print('Relative RMSE: ',rmse_rel)

Absolute RMSE:  332.0290754224323
Relative RMSE:  0.9485254690101602


## PLS Model

In this section, we train a PLS1 model with the data loaded in the section above.


### Select number of latent variables

Create a PLS model to predict the value of the final titer given the initial conditions.

* Input matrix: "DoE_nondim". This corresponds to the non-dimensional values of the manipulated variables for each experiment.
* Output target: "f_DoE". This corresponds to the final value of titer at the end of each experiment.

Select the number of latent variables for the model (the maximum number of latent variables is 5, equal to the number of variables in the input matrix).





In [101]:
# Number of latent variables
N_LV = 2

### Train the PLS model

Here we train the PLS1 model with final titer as our response

In [104]:
# Train PLS model
pls = PLSRegression(n_components=N_LV)
pls.fit(doe, y)

# Plot observed vs predicted
yhat = pls.predict(doe).reshape(len(y))
fig = px.scatter(x=y.values.reshape(len(yhat)),y=yhat,labels=dict(x="Observed Titer", y="Predicted Titer"), title="Observed vs Predicted on Train Set")
fig.update_layout(shapes = [{'type': 'line', 'yref': 'paper', 'xref': 'paper', 'y0': 0, 'y1': 1, 'x0': 0, 'x1': 1, 'layer': 'below'}])
fig.show()

### Plot variance explained in the Y

In [ ]:
# TODO: explained variance plot

### Evaluate model on test set


In [106]:
# Observed vs Predicted on Test Set
yhat_test = pls.predict(doe_test).reshape(len(y_test))
fig = px.scatter(x=y_test.values.reshape(len(yhat_test)),y=yhat_test,labels=dict(x="Observed Titer", y="Predicted Titer"), title="Observed vs Predicted on Train Set")
fig.update_layout(shapes = [{'type': 'line', 'yref': 'paper', 'xref': 'paper', 'y0': 0, 'y1': 1, 'x0': 0, 'x1': 1, 'layer': 'below'}])
fig.show()

In [108]:
# Absolute RMSE
rmse_abs = mean_squared_error(y_test, yhat_test,squared=False)
print('Absolute RMSE: ',rmse_abs)
# Relative RMSE
rmse_rel = mean_squared_error(y_test, yhat_test,squared=False) / np.std(y_test.values)
print('Relative RMSE: ',rmse_rel)

Absolute RMSE:  332.07526459136983
Relative RMSE:  0.9486574201143637


### Plot scores and loadings for the predictors


In [ ]:
# TODO: bi-plot / scores plot / loadings plot
# Color by exp name / FINAL TITER

### Plot VIP scores

In [ ]:
# TODO: plot VIP scores

### Task: What number of latent variables produces best model?

## Simulate Cross Validation

In this section, we will simulate a typical cross- validation to define the optimal number of latent variables.


### Select number of folds
First define the number of folds.


In [ ]:
# Number of folds
N_folds = 5

### Compute cross validation

Nfold PLS1 models are trained using (Nfolds-1) folds. For each model, the sum of squared residuals (SSR) is calculated and summed up.

This is repeated for different numbers of latent variables. The number of latent variables returning the least value of the SSRs is chosen as optimal.
A second criterion is selected, namely the Bayesian Information Criterion (BIC), which is weighting the effect of the number of latent variables, i.e., if two values of the number of latent variables are returning a similar value of the SSR, then the one using less variables is chosen to be more likely to produce robust predictions.:

### Evaluate model in cross-validation


# Historical PLS Model

In an so-called historical model, the data from different experiments are ordered into a batch-wise unfolded (BWU) matrix (i.e., every row corresponds to an experiment).

The BWU can be used to compute final properties of the experiment, like CQAs, which are typically the effect of the cumulated effect of the experiment profile.

In this example, we will use the BWU matrix to predict the final value of titer. Clearly, titer information are removed from the BWU matrix.

In [ ]:
# Define number of days to include
N_days = 13
# Define number of latent variables
N_LV = 5

### Create BWU matrix

Here the BWU matrix is created. The vaues of the manipulated variables are added as columns at the beginning of the matrix.

* Remove titer
* Remove exceeding days
* Eliminate invariant columns
* Remove linearly dependent columns
* Create a PLS model from the initial design to the final titer


In [ ]:
# TODO: code steps above

### Train Historical PLS model

In [ ]:
# TODO: train PLS on modified BWU

### Evaluate model on train/test set

In [ ]:
# TODO: plot obs vs predicted, compute abs/rel rmse

### Plot scores, loadings and VIP

In [ ]:
# TODO: plot bi-plot / scores / loadings / VIP values

### Task: Check model diagnostics

In [ ]:
# TODO: Residuals, mahalanobis distances

# Solutions to tasks

# Main Focus
* RMS model buildup
* Model comparison